In [ ]:
!pip install python-terrier
!pip install --upgrade git+https://github.com/terrierteam/pyterrier_t5.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.0/318.0 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.8/48.8 kB 6.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

In [1]:
#import pyterrier as pt
from tqdm import tqdm
#from pyterrier_t5 import MonoT5ReRanker
import pandas as pd
import gzip
import json
import os
#if not pt.started():
#  pt.init()
#monoT5 = MonoT5ReRanker(tok_model='t5-3b', model='castorini/monot5-3b-msmarco')

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
def already_covered_ids(dataset_name, query_type):
  ret = set()
  for f in ['', '-02', '-03', '-04']:
    f = f'/content/drive/MyDrive/{dataset_name}-{query_type}-with-scores-t5-3b-in-progress{f}.jsonl'
    df = pd.read_json(f, lines=True, dtype={'qid': str, 'docno': str})
    for _, i in df.iterrows():
      ret.add((i['qid'], i['docno']))

  return ret

def read_df(dataset_name, query_type, length=50000):
  src_file = f'/content/drive/MyDrive/{dataset_name}.jsonl.gz'
  df = []
  already_covered = already_covered_ids(dataset_name, query_type)
  skipped = 0
  with gzip.open(src_file, 'rt') as f:
    for l in tqdm(f, f'Load {src_file}.'):
      l = json.loads(l)
      if not l['qid'].endswith(f'_{query_type}'):
        continue

      if (l['qid'], l['docno']) in already_covered:
        skipped += 1
        continue

      if l['qid'].endswith(f'_{query_type}') and len(df) < length:
        df += [l]

  print(f'Did read {len(df)} lines, skipped {skipped}.')
  return pd.DataFrame(df)

read_df('tot-test-rerank', '1')

Load /content/drive/MyDrive/tot-test-rerank.jsonl.gz.: 750000it [00:33, 22377.35it/s]

Did read 0 lines, skipped 150000.


""


In [20]:
def rerank_dataset(dataset_name, query_type, length=50000):
  target_file = f'/content/drive/MyDrive/{dataset_name}-{query_type}-with-scores-t5-3b-in-progress-04.jsonl'
  df = read_df(dataset_name, query_type, length)

  run = monoT5.transform(df)
  del run['query']
  del run['text']

  run.to_json(target_file, lines=True, orient='records')

In [21]:
for dataset in ['tot-test-rerank']:
    for method in ['1']:
        rerank_dataset(dataset, method)

Load /content/drive/MyDrive/tot-test-rerank.jsonl.gz.: 750000it [00:24, 30773.61it/s]


Did read 40000 lines, skipped 110000.


monoT5:   0%|          | 0/10000 [00:00<?, ?batches/s]

In [22]:
print("Hello world")

Hello world


In [ ]:
def rerank_dataset(dataset_name, query_type, length=50000):
  src_file = f'/content/drive/MyDrive/{dataset_name}.jsonl.gz'
  target_file = f'/content/drive/MyDrive/{dataset_name}-{query_type}-with-scores-t5-3b-in-progress-02.jsonl'

  try:
    pd.read_json(target_file, lines=True)
    return
  except:
    pass

  df = []
  skipped = 0
  with gzip.open(src_file, 'rt') as f:
    for l in tqdm(f, f'Load {src_file}.'):
      l = json.loads(l)
      if skipped < (2*length):
        skipped += 1
        continue
      if l['qid'].endswith(f'_{query_type}') and len(df) < length:
        df += [l]
  df = pd.DataFrame(df)

  run = monoT5.transform(df)
  del run['query']
  del run['text']

  run.to_json(target_file, lines=True, orient='records')

In [ ]:
for dataset in ['tot-test-rerank']:
    for method in ['1']:
        rerank_dataset(dataset, method)